In [1]:
require 'nn'
require 'optim'

torch.manualSeed(0)
torch.setnumthreads(4)

In [2]:
function setup() 
    classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
    geometry = { 32, 32 }

    net = nn.Sequential()

    net:add(nn.SpatialConvolution(1, 6, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.SpatialConvolution(6, 16, 5, 5))
    net:add(nn.ReLU())
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.View(16*5*5))
    net:add(nn.Linear(16*5*5, 120))
    net:add(nn.ReLU())
    net:add(nn.Linear(120, 84))
    net:add(nn.ReLU())
    net:add(nn.Linear(84, #classes))
    net:add(nn.LogSoftMax())
    
    parameters, gradParameters = net:getParameters()
    criterion = nn.ClassNLLCriterion()
    confusion = optim.ConfusionMatrix(classes)
end

In [3]:
function normalize(input, n_channels)
    local mean = {}
    local stdev = {}
    
    for channel = 1, n_channels do
        mean[channel] = input.data[{ {}, {channel}, {}, {} }]:mean()
        stdev[channel] = input.data[{ {}, {channel}, {}, {} }]:std()
        
        print('Channel ' .. channel .. ' mean: ' .. mean[channel] .. ' stdev: ' .. stdev[channel])
        
        input.data[{ {}, {channel}, {}, {} }]:add(-mean[channel])
        input.data[{ {}, {channel}, {}, {} }]:div(stdev[channel])
    end
end

In [4]:
function load_data(double_training, test_method)    
    train = torch.load('mnist-p1b-train.t7')
    test = torch.load('mnist-p1b-test.t7')
    
    n_train = train.data:size()[1]
    n_test = test.data:size()[1]
    
    if double_training then
        train.data:resize(n_train * 2, 1, geometry[1], geometry[2])
        train.label:resize(n_train * 2)
        for i = 1, n_train do
            train.data[n_train + i] = 255 - train.data[i]
            train.label[n_train + i] = train.label[i]
        end
        n_train = n_train * 2
    end
    
    if test_method == 'invert' then
        for i = 1, n_test do
            test.data[i] = 255 - test.data[i]
        end
    elseif test_method == 'double' then
        test.data:resize(n_test * 2, 1, geometry[1], geometry[2])
        test.label:resize(n_test * 2)
        for i = 1, n_test do
            test.data[n_test + i] = 255 - test.data[i]
            test.label[n_test + i] = test.label[i]
        end
        n_test = n_test * 2
    end
    
    train.data = train.data:double()
    test.data = test.data:double()
end

In [5]:
function exec_training(obj, n_channels, current_epoch) 
    confusion:zero()
    
    for t = 1, n_train, batch_size do
        local limit = math.min(t + batch_size - 1, n_train)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local feval = function(x)
            collectgarbage()
            
            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()
            
            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)

            for i = 1, limited_batch_size do
                confusion:add(outputs[i], targets[i])
            end
            
            return f, gradParameters
        end
        
        sgd_state = sgd_state or {
            learningRate = 0.03,
            learningRateDecay = 1e-7,
            momentum = 0.5
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    confusion:updateValids()
    return confusion.averageValid
end

In [6]:
function exec_testing(obj, n_channels, current_epoch)
    confusion:zero()
    
    for t = 1, n_test, batch_size do
        local limit = math.min(t + batch_size - 1, n_test)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local preds = net:forward(inputs)
        
        for i = 1, limited_batch_size do
            confusion:add(preds[i], targets[i])
        end
    end 
    
    confusion:updateValids()
    return confusion.averageValid
end

In [8]:
setup()
load_data(false, 'invert')
normalize(train, 1)
normalize(test, 1)

n_epoch = 32
batch_size = 256
local last_train_conf
local last_test_conf
for epoch = 1, n_epoch do
    local acc_train = exec_training(train, 1, epoch)
    
    if (epoch == n_epoch) then
        last_train_conf = confusion:__tostring__()
    end
    
    local acc_test = exec_testing(test, 1, epoch)
    
    if (epoch == n_epoch) then
        last_test_conf = confusion:__tostring__()
    end
    
    io.write(string.format('Epoch %3d: %.4f%% | %.4f%%\n', epoch, acc_train * 100, acc_test * 100))
end

print(last_train_conf)
print(last_test_conf)

Channel 1 mean: 25.509416422526 stdev: 70.180423838273	


Channel 1 mean: 229.12859375 stdev: 70.739298703524	


Epoch   1: 50.9522% | 0.8196%


Epoch   2: 91.0095% | 9.1345%


Epoch   3: 94.9684% | 10.8786%


...

Epoch  28: 99.4481% | 31.1699%


Epoch  29: 99.4830% | 31.1294%


Epoch  30: 99.5260% | 31.1702%


Epoch  31: 99.5546% | 31.2417%


Epoch  32: 99.5780% | 31.2405%
ConfusionMatrix:
[[    5914       1       2       0       0       0       2       0       1       3]   99.848% 	[class: 1]
 [       1    6721       2       0       1       0       2      10       4       1]   99.689% 	[class: 2]
 [       2       3    5946       1       1       0       0       4       1       0]   99.799% 	[class: 3]
 [       1       1       4    6101       0       7       0       6       6       5]   99.511% 	[class: 4]
 [       0       3       2       0    5816       0       5       2       1      13]   99.555% 	[class: 5]
 [       2       1       1       5       1    5395       8       0       5       3]   99.520% 	[class: 6]
 [       1       2       1       0       2       6    5902       0       4       0]   99.730% 	[class: 7]
 [       0       9       3       1       1       0       0    6238       6       7]   99.569% 	[class: 8]
 [       4       6       1       7       3       3       4       2    5813       8]   99.351% 	[class: 9

In [9]:
load_data(true, 'double')
normalize(train, 1)
normalize(test, 1)

for epoch = 1, n_epoch do
    local acc_train = exec_training(train, 1, epoch, true)
    
    if (epoch == n_epoch) then
        last_train_conf = confusion:__tostring__()
    end
    
    local acc_test = exec_testing(test, 1, epoch, 'both')
    
    if (epoch == n_epoch) then
        last_test_conf = confusion:__tostring__()
    end
    
    io.write(string.format('Epoch %3d: %.4f%% | %.4f%%\n', epoch, acc_train * 100, acc_test * 100))
end

print(last_train_conf)
print(last_test_conf)

Channel 1 mean: 127.5 stdev: 123.80376034192	


Channel 1 mean: 127.5 stdev: 123.82414832818	


Epoch   1: 96.6630% | 89.4073%


Epoch   2: 97.7767% | 96.1924%


Epoch   3: 98.3195% | 97.4384%


...

Epoch  28: 99.7865% | 98.7105%


Epoch  29: 99.8025% | 98.7209%


Epoch  30: 99.8210% | 98.7511%


Epoch  31: 99.8403% | 98.7521%


Epoch  32: 99.8549% | 98.7477%
ConfusionMatrix:
[[   11843       0       0       0       0       0       2       0       1       0]   99.975% 	[class: 1]
 [       0   13465       1       0       2       1       1      13       1       0]   99.859% 	[class: 2]
 [       0       3   11909       0       0       0       1       2       1       0]   99.941% 	[class: 3]
 [       0       1       1   12246       0       2       0       3       5       4]   99.870% 	[class: 4]
 [       0       3       0       0   11660       0       3       3       1      14]   99.795% 	[class: 5]
 [       0       0       0       5       0   10828       4       0       3       2]   99.871% 	[class: 6]
 [       3       4       2       0       2       6   11813       0       6       0]   99.806% 	[class: 7]
 [       0      12       1       1       0       0       0   12512       0       4]   99.856% 	[class: 8]
 [       0       5       1       3       0       2       3       1   11682       5]   99.829% 	[class: 9